<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance==0.2.59
!pip install ta

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSI_14', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%', 'ROE%/PE']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-06-25 11:39:42
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


get_prospects_features
qualified stocks: 282
with latest results: 78
still star stocks: 56


In [4]:
df_prospects_features

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,Today%,FTT%,...,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%,ROE%/PE
0,3MINDIA,28500.00,40020.60,-7.20,-15.12,H-MC,37.0,12.10,0.30,40.42,...,ATH,13.21,-11.14,MISC,NaN,33095.37,OX40N,22.58,42.28,0.3
1,5PAISA,412.75,565.00,-5.33,152.40,H-SC,58.0,60.85,2.58,36.89,...,NTT,39.09,9.46,FINANCE,1.0,1148.55,OX40N,10.29,-85.27,0.5
2,AARTIIND,470.10,919.00,2.87,34.78,M-SC,56.0,43.42,3.50,95.49,...,NTT,14.10,-21.62,CHEMICALS,NaN,17292.16,XR,6.04,122.49,0.1
3,AAVAS,1917.60,2328.00,7.79,-30.34,M-SC,59.0,83.27,1.31,21.40,...,NTT,16.70,17.06,FINANCE,NaN,15206.68,XY24,14.12,-93.45,0.5
4,ABB,5996.00,7934.00,-6.87,-34.51,H-LC,54.0,38.26,-0.12,32.32,...,NTT,10.36,29.16,ELECTRICAL,1.0,128235.29,AR,28.82,49.98,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,WHIRLPOOL,1385.00,2270.00,-11.33,-38.30,M-SC,63.0,83.63,0.76,63.90,...,NTT,15.95,54.41,DURABLES,1.0,16333.30,XR/OX40,9.27,77.06,0.2
371,WIPRO,267.80,420.00,-2.26,-2.86,M-LC,68.0,77.58,1.07,56.83,...,NTT,-0.75,20.20,IT,1.0,260542.41,XR/X5K,16.59,77.47,0.8
372,WSTCSTPAPR,508.10,772.47,-2.29,NaN,NaN,57.0,51.96,-0.17,52.03,...,ATH,NaN,NaN,PAPER,NaN,NaN,XR,NaN,NaN,NaN
373,YESBANK,20.05,388.67,0.25,-45.09,L-MC,48.0,14.23,0.40,1838.50,...,NaN,12.00,90.13,BANKS,NaN,65387.34,NaN,5.44,25.32,0.2


In [5]:
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSI_14', 'RSP', 'FTT%', 'ATH%','Gained%', 'CumlRnk','ROE%/PE', 'Criteria', 'Strategy', 'Category', 'InFolio']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
list_ox40 = list(tmp_df[tmp_df['MBQ'].str.contains('OX40', na=False)]['Symbol'].values)
list_ox40n = list(tmp_df[tmp_df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list(set(list_x40 + list_x5k + list_hc)- set(list_ox40 + list_ox40n))))

In [6]:
# get the cumulative rank
df_tmp = df_prospects_features # [df_prospects_features['Symbol'].isin(stock_list)]

df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)
df_tmp['Rnk_FTT%'] = df_tmp['FTT%'].rank(method='max', ascending=False)
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_RSI_14'] = df_tmp['RSI_14'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%/PE'] = df_tmp['ROE%/PE'].rank(method='max', ascending=False)
df_tmp['Ovrl_Rank'] = df_tmp['Rnk_Dev%_200'] + df_tmp['Rnk_Dev%_PE'] + df_tmp['Rnk_Gained%'] + df_tmp['Rnk_FTT%'] + df_tmp['Rnk_Sales_Grwth%'] + df_tmp['Rnk_Profit_Grwth%'] + df_tmp['Rnk_MCap'] + df_tmp['Rnk_CFO_2_EBITDA%'] + df_tmp['Rnk_ROE%'] + df_tmp['Rnk_RSI_14'] + df_tmp['Rnk_ROE%/PE']
df_tmp['Cat_Rank'] = df_tmp.groupby('Conviction')['Ovrl_Rank'].rank(ascending=True, method='min')
priority_map = {'X-LC': 0,'H-LC': 1,'X-MC': 2,'X-SC': 3,'M-LC': 4,'H-MC': 5,'H-SC': 6,'L-LC': 7,'M-MC': 8,'M-SC': 9,'L-MC': 10,'L-SC': 11}
df_tmp['Conviction_Priority'] = df_tmp['Conviction'].map(priority_map)
df_tmp = df_tmp.sort_values(by=['Conviction_Priority', 'Cat_Rank']).reset_index(drop=True)
df_tmp['CumlRnk'] = df_tmp.index + 1
# Drop the helper columns
df_tmp.drop(columns=['Rnk_Dev%_200','Rnk_Dev%_PE','Rnk_Gained%','Rnk_FTT%','Rnk_Sales_Grwth%','Rnk_Profit_Grwth%','Rnk_MCap','Rnk_ROE%','Rnk_RSI_14','Rnk_CFO_2_EBITDA%'], inplace=True)
df_prospects_features = df_tmp[cols]
data_table.DataTable(df_prospects_features.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4998.00,-10.89,-18.06,X-LC,47.0,23.84,46.33,29.76,6.63,1,2.1,X40,BTT,IT,1.0
1,INFY,2275.00,-7.53,-10.62,X-LC,55.0,56.58,41.88,22.98,16.42,2,1.2,X40,BTT,IT,1.0
2,BAJFINANCE,9150.00,16.73,-90.98,X-LC,53.0,42.17,889.72,3.92,44.03,3,5.5,X40,BTT,FINANCE,NaN
3,BAJAJ-AUTO,12353.32,-5.62,47.06,X-LC,54.0,16.73,47.31,47.31,17.68,4,0.7,X40,ATH,AUTO,NaN
4,SUNPHARMA,1937.03,-5.20,17.67,X-LC,47.0,35.94,15.76,15.76,12.19,5,0.5,NaN,ATH,PHARMA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,SPORTKING,157.26,6.70,NaN,NaN,50.0,8.36,43.09,43.09,53.32,279,NaN,AR,ATH,TEXTILES,NaN
279,TASTYBITE,20387.67,4.92,NaN,NaN,61.0,58.01,86.05,86.05,40.75,280,NaN,XR,ATH,FMCG,NaN
280,V2RETAIL,2031.65,19.78,NaN,NaN,57.0,13.52,7.28,7.28,173.06,281,NaN,AR,ATH,TEXTILES,NaN
281,VEEDOL,3065.39,-2.63,NaN,NaN,54.0,40.75,83.79,83.79,22.94,282,NaN,XR,ATH,CHEMICALS,NaN


In [7]:
# module to print date timestamp
def read_csv(csv_file_name):
  csv_file_path = os.path.join(project_path, 'data', csv_file_name)
  df = pd.read_csv(csv_file_path)
  return df

mypps_df = read_csv('myProspectsScrips.csv')

# Suppose this is your updated DataFrame with new cuml_rnk values
cols1 = ['Symbol', 'CumlRnk']
df_tmp = df_prospects_features[cols1]

# Merge the two DataFrames on 'symbol', keeping the CSV structure
mypps_df = mypps_df.merge(df_tmp[['Symbol', 'CumlRnk']], on='Symbol', how='left', suffixes=('', '_new'))

# Replace 'cuml_rnk' with the new value only if it exists in df
mypps_df['CumlRnk'] = mypps_df['CumlRnk_new'].combine_first(mypps_df['CumlRnk'])

# Drop the helper column
mypps_df.drop(columns=['CumlRnk_new'], inplace=True)

# Save back to CSV
#mypps_df.to_csv(project_path+'/data/myProspectsScrips.csv', index=False)

In [8]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]
df_tmp = df_tmp[(df_tmp['FTT%']>20) & (df_tmp['Dev%_PE']<0)]
data_table.DataTable(df_tmp.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4998.00,-10.89,-18.06,X-LC,47.0,23.84,46.33,29.76,6.63,1,2.1,X40,BTT,IT,1.0
1,INFY,2275.00,-7.53,-10.62,X-LC,55.0,56.58,41.88,22.98,16.42,2,1.2,X40,BTT,IT,1.0
2,BAJFINANCE,9150.00,16.73,-90.98,X-LC,53.0,42.17,889.72,3.92,44.03,3,5.5,X40,BTT,FINANCE,NaN
11,SIEMENS,7969.85,-43.09,-7.62,X-LC,40.0,17.08,154.97,154.97,15.09,12,0.4,AR,ATH,ELECTRICAL,1.0
14,DMART,5201.00,5.74,-16.71,X-LC,59.0,57.65,21.87,26.33,25.37,15,0.1,X40N,NTT,FMCG,1.0
24,VBL,672.28,-16.92,-17.97,H-LC,40.0,13.88,45.28,45.28,6.32,25,0.4,X40N,ATH,FMCG,1.0
27,ITC,505.32,-4.74,-35.32,H-LC,48.0,16.37,21.11,21.11,7.62,28,1.8,X40,ATH,FMCG,NaN
28,ABB,7934.00,-6.87,-34.51,H-LC,54.0,38.26,32.32,49.32,22.23,29,0.4,AR,NTT,ELECTRICAL,1.0
31,ASIANPAINT,4250.00,-7.93,-21.74,H-LC,54.0,33.10,85.91,51.60,8.21,32,0.3,X40,BTT,PAINTS,1.0
41,BAJAJHFL,152.00,-4.18,-3.84,H-LC,49.0,27.05,24.91,49.15,15.08,42,0.3,X40N,BTT,FINANCE,1.0


In [9]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
189,QUESS,986.0,-48.97,-33.33,M-SC,31.0,3.56,227.52,297.69,0.96,190,0.9,XY24,NTT,MISC,1.0
188,EASEMYTRIP,26.4,-24.00,7.71,M-SC,39.0,14.59,148.12,221.43,2.50,189,0.7,XY24,NTT,TRAVEL,1.0
128,BANDHANBNK,400.0,8.86,-29.05,H-SC,60.0,86.48,120.02,292.19,39.51,129,1.1,XY24,NTT,BANKS,1.0
166,ACC,3906.0,-10.32,-37.20,M-MC,46.0,12.81,110.61,48.88,2.93,167,0.9,XY24,BTT,CEMENT,1.0
212,GREENPANEL,537.0,-15.36,193.93,M-SC,57.0,56.94,99.52,122.81,23.55,213,0.2,XY24,NTT,MISC,1.0
132,REPCOHOME,880.0,4.53,-52.31,H-SC,66.0,91.81,98.56,90.26,42.39,133,2.3,XY24,NTT,FINANCE,1.0
97,AWL,485.0,-11.39,-69.30,H-MC,44.0,37.01,87.44,225.02,9.76,98,0.5,XY24,NTT,FMCG,1.0
229,ATULAUTO,844.0,-15.16,3348.65,M-SC,44.0,5.69,85.25,75.53,9.82,230,0.0,XY24,NTT,AUTO,1.0
224,SHALBY,327.0,-16.95,831.42,M-SC,45.0,2.85,78.96,76.05,1.48,225,0.1,XY24,NTT,HEALTHCARE,1.0
214,STARHEALTH,761.0,-6.21,-4.22,M-SC,44.0,5.34,76.38,110.19,25.75,215,0.2,XY24,NTT,INSURANCE,1.0


In [10]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
60,VAIBHAVGBL,521.00,-10.94,53.72,X-SC,43.0,2.49,120.76,295.56,17.65,61,0.4,XR,NTT,JEWELLERY,1.0
64,ALKYLAMINE,4546.37,21.61,6.76,X-SC,83.0,98.58,96.86,96.86,47.91,65,0.2,SR,ATH,CHEMICALS,1.0
62,DEEPAKNTR,3102.60,-17.51,17.79,X-SC,47.0,11.03,60.60,60.60,6.89,63,0.4,AR,ATH,CHEMICALS,NaN
51,COLPAL,3767.14,-12.40,0.55,X-MC,46.0,26.69,54.92,54.92,4.58,52,1.8,XY25,ATH,FMCG,1.0
54,DABUR,735.00,-7.87,-13.64,X-MC,53.0,37.37,53.12,38.36,6.34,55,0.4,XY24,BTT,FMCG,1.0
55,HONAUT,58357.33,-3.89,-19.55,X-MC,60.0,63.35,50.91,50.91,18.77,56,0.2,X40N,ATH,ELECTRICAL,1.0
53,PGHH,18106.42,-8.28,-19.82,X-MC,43.0,12.46,34.22,34.22,5.47,54,1.3,X40,ATH,FMCG,1.0
63,MEDANTA,1486.00,-0.54,2.01,X-SC,37.0,6.05,31.75,31.84,16.65,64,0.3,XY24,NTT,HEALTHCARE,1.0
52,HAVELLS,2069.16,-3.02,-4.04,X-MC,59.0,49.11,30.09,30.09,13.47,53,0.3,X40,ATH,ELECTRICAL,1.0
67,CERA,8422.00,-0.38,-14.80,X-SC,55.0,63.70,24.37,53.89,33.23,68,0.5,OX40N,NTT,CERAMICS,1.0


In [11]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
126,TANLA,1963.11,2.64,-30.09,H-SC,61.0,93.95,200.26,200.26,56.51,127,1.4,AR,ATH,IT,1.0
133,RELAXO,1176.00,-27.66,-41.32,H-SC,42.0,9.25,183.72,243.41,3.65,134,0.1,X40N,NTT,FOOTWEAR,1.0
128,BANDHANBNK,400.00,8.86,-29.05,H-SC,60.0,86.48,120.02,292.19,39.51,129,1.1,XY24,NTT,BANKS,1.0
122,SONATSOFTW,852.37,-18.62,-0.11,H-SC,54.0,59.79,106.59,106.59,37.03,123,1.0,MGC,ATH,IT,NaN
132,REPCOHOME,880.00,4.53,-52.31,H-SC,66.0,91.81,98.56,90.26,42.39,133,2.3,XY24,NTT,FINANCE,1.0
144,VALIANTORG,838.00,22.54,-362.01,H-SC,52.0,79.36,95.38,331.49,86.93,145,0.0,XR,NTT,CHEMICALS,1.0
147,GLAND,3442.00,8.07,7.06,H-SC,73.0,95.73,93.38,140.02,27.79,148,0.2,XR,NTT,PHARMA,NaN
97,AWL,485.00,-11.39,-69.30,H-MC,44.0,37.01,87.44,225.02,9.76,98,0.5,XY24,NTT,FMCG,1.0
120,INDIAMART,4850.92,12.26,-46.40,H-SC,75.0,93.59,84.11,84.11,38.53,121,0.9,AR,ATH,MISC,1.0
130,METROPOLIS,3102.00,-9.85,-0.91,H-SC,52.0,55.16,82.26,101.63,14.34,131,0.2,XR,NTT,HEALTHCARE,NaN


In [12]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
2,BAJFINANCE,9150.00,16.73,-90.98,X-LC,53.0,42.17,889.72,3.92,44.03,3,5.5,X40,BTT,FINANCE,NaN
31,ASIANPAINT,4250.00,-7.93,-21.74,H-LC,54.0,33.10,85.91,51.60,8.21,32,0.3,X40,BTT,PAINTS,1.0
205,BATAINDIA,2096.00,-7.98,-32.87,M-SC,45.0,9.96,73.22,79.81,2.46,206,0.3,X40,NTT,FOOTWEAR,1.0
3,BAJAJ-AUTO,12353.32,-5.62,47.06,X-LC,54.0,16.73,47.31,47.31,17.68,4,0.7,X40,ATH,AUTO,NaN
0,TCS,4998.00,-10.89,-18.06,X-LC,47.0,23.84,46.33,29.76,6.63,1,2.1,X40,BTT,IT,1.0
1,INFY,2275.00,-7.53,-10.62,X-LC,55.0,56.58,41.88,22.98,16.42,2,1.2,X40,BTT,IT,1.0
53,PGHH,18106.42,-8.28,-19.82,X-MC,43.0,12.46,34.22,34.22,5.47,54,1.3,X40,ATH,FMCG,1.0
52,HAVELLS,2069.16,-3.02,-4.04,X-MC,59.0,49.11,30.09,30.09,13.47,53,0.3,X40,ATH,ELECTRICAL,1.0
137,SANOFI,7552.00,0.30,65.96,H-SC,46.0,19.93,24.64,63.28,23.70,138,1.3,X40,NTT,PHARMA,NaN
118,ICICIPRULI,790.00,-1.88,-10.84,H-MC,53.0,25.27,24.06,24.06,18.83,119,0.1,X40,ATH,INSURANCE,1.0


In [13]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
133,RELAXO,1176.00,-27.66,-41.32,H-SC,42.0,9.25,183.72,243.41,3.65,134,0.1,X40N,NTT,FOOTWEAR,1.0
207,TEAMLEASE,3487.00,-18.21,165.44,M-SC,52.0,48.04,74.59,164.39,17.83,208,0.4,X40N,NTT,MISC,NaN
55,HONAUT,58357.33,-3.89,-19.55,X-MC,60.0,63.35,50.91,50.91,18.77,56,0.2,X40N,ATH,ELECTRICAL,1.0
24,VBL,672.28,-16.92,-17.97,H-LC,40.0,13.88,45.28,45.28,6.32,25,0.4,X40N,ATH,FMCG,1.0
121,BLUESTARCO,2337.55,-13.45,-12.48,H-SC,53.0,71.00,42.45,42.45,7.09,122,0.4,X40N,ATH,AC,NaN
151,KAJARIACER,1541.71,3.12,22.09,H-SC,73.0,73.31,38.31,38.31,43.01,152,0.2,X40N,ATH,CERAMICS,NaN
98,DIXON,18940.15,-4.19,-36.93,H-MC,46.0,8.90,30.57,30.57,33.01,99,0.4,X40N,ATH,IT,1.0
124,LALPATHLAB,3545.00,-4.41,-33.70,H-SC,42.0,21.00,27.79,47.87,20.32,125,0.5,X40N,NTT,HEALTHCARE,1.0
83,GODREJCP,1495.48,-1.87,15.16,M-LC,38.0,3.91,26.89,26.89,19.48,84,0.2,X40N,ATH,FMCG,NaN
41,BAJAJHFL,152.00,-4.18,-3.84,H-LC,49.0,27.05,24.91,49.15,15.08,42,0.3,X40N,BTT,FINANCE,1.0


In [14]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
265,BAJAJHIND,459.18,-14.14,NaN,NaN,55.0,40.75,1810.07,1810.07,37.29,266,NaN,XR,ATH,SUGAR,NaN
247,CENTRALBK,190.84,-19.00,-38.15,L-MC,54.0,64.77,390.84,390.84,14.83,248,1.3,XR,ATH,BANKS,NaN
262,ALOKINDS,91.59,-1.30,NaN,L-SC,55.0,72.60,363.98,363.98,32.66,263,NaN,AR,ATH,TEXTILES,NaN
263,APEX,849.16,-0.01,NaN,NaN,57.0,74.73,261.38,261.38,19.43,264,NaN,AR,ATH,MISC,NaN
204,IGL,707.00,0.71,-8.94,M-SC,53.0,46.98,239.12,36.13,33.90,205,1.0,AR,BTT,GAS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,MAXHEALTH,1246.80,14.48,50.53,X-LC,58.0,65.48,1.79,1.79,43.61,22,0.1,AR,NaN,HEALTHCARE,NaN
49,GRASIM,2832.86,7.86,165.36,H-LC,66.0,69.04,1.25,1.25,22.28,50,0.1,AR,ATH,TEXTILES,NaN
58,SRF,3126.30,16.83,97.41,X-MC,59.0,75.80,1.20,1.20,44.28,59,0.1,XR,ATH,CHEMICALS,NaN
77,INDIGO,5695.00,19.01,28.04,M-LC,60.0,62.99,1.08,1.08,46.38,78,4.3,AR,ATH,AIRLINES,NaN
